# Exploratory coding for I3D
## Includes code snippets to transform input data into format compatible with a pretrained I3D model from DeepMind

In [6]:
!ls ./data

label_map.txt			v_CricketShot_g04_c01_rgb.gif
v_CricketShot_g04_c01_flow.gif	v_CricketShot_g04_c01_rgb.npy
v_CricketShot_g04_c01_flow.npy


In [7]:
import numpy as np

Input is of dimension (batch x frames x axis1 x axis2 x channels)

In [8]:
flow_matrix=np.load('./data/v_CricketShot_g04_c01_flow.npy')

In [9]:
flow_matrix.shape

(1, 79, 224, 224, 2)

In [10]:
rgb_matrix=np.load('./data/v_CricketShot_g04_c01_rgb.npy')

In [11]:
rgb_matrix.shape

(1, 79, 224, 224, 3)

In [13]:
!python evaluate_sample.py

Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
2019-06-10 16:00:30.804895: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2019-06-10 16:00:30.826687: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2808000000 Hz
2019-06-10 16:00:30.827320: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55aa848cb440 executing computations on platform Host. Devices:
2019-06-10 16:00:30.827377: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-06-10 16:00:30.889422: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-06-10 16:00:30.889917: I tensorflow/compiler/xla/service/service.cc:150] XLA servic

## Need to extract numpy frames from video clips using OpenCV

## Generates two outputs:

### 1) RGB output

### 2) TV-L1 Optical Flow output (2 channels)

In [16]:
import cv2
import os
import numpy as np

In [17]:
!ls ../SoccerNet-code/data/

england_epl		      README.md
europe_uefa-champions-league  SoccerNet_V1.1_Commentaries.csv
france_ligue-1		      SoccerNet_V1.1_Features.csv
germany_bundesliga	      SoccerNet_V1.1_Labels.csv
goals			      SoccerNet_V1.1_Videos.csv
italy_serie-a		      spain_laliga
nongoals


In [78]:
# code code snippet for TVL1 computation between frames

def compute_TVL1(prev, curr, bound=15):
    """Compute the TV-L1 optical flow."""
    TVL1 = cv2.DualTVL1OpticalFlow_create()
    flow = TVL1.calc(prev, curr, None)
    
    # clip to range [-20, 20]
    np.clip(flow, -20, 20, out=flow)
    
    # normalize between -1 and 1
    flow = flow/20

    return flow

In [ ]:
# Video generator code for data input





In [46]:
# test code to see whether frame retrieval and flow calculation works as intended
list_of_videos = os.listdir('../SoccerNet-code/data/goals')[0]
vid = os.path.join('../SoccerNet-code/data/goals', list_of_videos)
cap = cv2.VideoCapture(vid)
_, frame1 = cap.read()
gray1 = cv2.cvtColor(frame1, cv2.COLOR_RGB2GRAY)


_, frame2 = cap.read()
gray2 = cv2.cvtColor(frame2, cv2.COLOR_RGB2GRAY)

flow = compute_TVL1(gray1, gray2, None)
cv2.imwrite('./' + 'original' + '.png', frame2)
cv2.imwrite('./' + 'flow' + '.png', flow_vis)

error: OpenCV(3.4.2) /tmp/build/80754af9/opencv-suite_1535558553474/work/modules/imgproc/src/color.hpp:253: error: (-215:Assertion failed) VScn::contains(scn) && VDcn::contains(dcn) && VDepth::contains(depth) in function 'CvtHelper'


In [66]:
frame2.min()

0

In [76]:
flow.nbytes

713216

In [75]:
frame2.nbytes

267456

In [25]:
!ls

data		    I3D_exploratory.ipynb  original.png
evaluate_sample.py  i3d_inception.py	   __pycache__
flow.png	    LICENSE		   README.md


In [47]:
!ls ./data

label_map.txt			v_CricketShot_g04_c01_rgb.gif
v_CricketShot_g04_c01_flow.gif	v_CricketShot_g04_c01_rgb.npy
v_CricketShot_g04_c01_flow.npy


In [48]:
flow_matrix = np.load('./data/v_CricketShot_g04_c01_flow.npy')
flow_matrix.shape

(1, 79, 224, 224, 2)

In [64]:
flow_matrix.min()

-0.46697944

In [55]:
rgb_matrix = np.load('./data/v_CricketShot_g04_c01_rgb.npy')
rgb_matrix.shape

(1, 79, 224, 224, 3)

In [59]:
rgb_matrix.min()

-1.0

In [61]:
flow.min()

-2.333117

In [79]:
def vid2npy(filename):
    cap = cv2.VideoCapture(filename)
    frames =[]
    while True:
        ret, frame = cap.read()
        if ret == True:
            frame = cv2.resize(frame, (224,224), interpolation=cv2.INTER_AREA)
            frames.append(frame)
        else:
            break
            
    return np.array(frames)

In [80]:
clip = vid2npy('../SoccerNet-code/data/goals/1.mkv')

In [81]:
clip.shape

(250, 224, 224, 3)

In [16]:
def get_frames(video_folder='./data/goals', cls_label=1):

    X_data = []
    X_flow = []
    y_data = []
    list_of_videos = os.listdir(video_folder)[:10]

    for i in list_of_videos:
        #Video Path
        vid = os.path.join(video_folder, i) #path to each video from list1 = os.listdir(path)
        #Reading the Video
        cap = cv2.VideoCapture(vid)
        
        #Reading Frames
        #fps = vcap.get(5)
        #To Store Frames
        frames = []
        while True: #here we get 40 frames, for example
            ret, frame = cap.read()
            if ret == True:
                frame = cv2.resize(frame,(224,224),interpolation=cv2.INTER_AREA)
                frames.append(frame)
            else:
                print('Class 1 - Success!')
                break
        X_data.append(frames) #appending each tensor of 40 frames resized to 30x30
        y_data.append(cls_label) #appending a class label to the set of 40 frames
    X_data = np.array(X_data)
    y_data = np.array(y_data) #ready to split! :)

    return X_data, y_data

In [17]:
X_train, y_train = get_frames('../SoccerNet-code/data/goals', 1)



Class 1 - Success!
Class 1 - Success!
Class 1 - Success!
Class 1 - Success!
Class 1 - Success!
Class 1 - Success!
Class 1 - Success!
Class 1 - Success!
Class 1 - Success!
Class 1 - Success!


In [18]:
X_train.shape

(10, 250, 224, 224, 3)

In [3]:
from i3d_inception import Inception_Inflated3d
from i3d_inception import conv3d_bn

from keras import backend as K

from keras.models import Model
from keras.layers import Activation
from keras.layers import Add
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import Lambda

Using TensorFlow backend.


In [10]:
rgb_model = Inception_Inflated3d(
    include_top=False,
    weights='rgb_imagenet_and_kinetics',
    input_shape=(250, 224, 224, 3))

In [11]:
rgb_model.input

<tf.Tensor 'input_5:0' shape=(?, 250, 224, 224, 3) dtype=float32>

In [13]:
# combine Inflated3d-conv3d outputs from RGB and flow

def generate_logit(x, last_conv3d_name, classes):
    
    x = conv3d_bn(x, classes, 1, 1, 1, padding='same',
                   use_bias=True, use_activation_fn=False, use_bn=False, name=last_conv3d_name)

    num_frames_remaining = int(x.shape[1])
    x = Reshape((num_frames_remaining, classes))(x)

    x = Lambda(lambda x: K.mean(x, axis=1, keepdims=False),
               output_shape=lambda s: (s[0], s[2]))(x)    
    
    return x

def Combined_I3D_model(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS, NUM_FLOW_CHANNELS, NUM_CLASSES, dropout_prob):

    rgb_model = Inception_Inflated3d(
        include_top=False,
        weights='rgb_imagenet_and_kinetics',
        input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_RGB_CHANNELS))

    flow_model = Inception_Inflated3d(
        include_top=False,
        weights='flow_imagenet_and_kinetics',
        input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_FLOW_CHANNELS))

    x1 = rgb_model.layers[-1].output
    x1 = Dropout(dropout_prob)(x1)
    
    x2 = flow_model.layers[-1].output
    x2 = Dropout(dropout_prob)(x2)

    # generate logits for RGB and flow streams
    x1 = generate_logit(x1, '1x1_Conv3d_rgb_logits', NUM_CLASSES)
    x2 = generate_logit(x2, '1x1_Conv3d_rgb_logits', NUM_CLASSES)
    
    x = Add()([x1, x2])
    x = Activation('softmax', name='prediction')(x)

    combined_model = Model(input=[rgb_model.input, flow_model.input], output=x)
    
    return combined_model

In [14]:
full_model = Combined_I3D_model(NUM_FRAMES=250,
                                FRAME_HEIGHT=224,
                                FRAME_WIDTH=224,
                                NUM_RGB_CHANNELS=3,
                                NUM_FLOW_CHANNELS=2,
                                NUM_CLASSES=2,
                                dropout_prob=0.0)

/home/dshin1331/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("pr...)`


ValueError: The name "Conv3d_1a_7x7_conv" is used 2 times in the model. All layer names should be unique.

In [27]:
full_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 250, 224, 224 0                                            
__________________________________________________________________________________________________
Conv3d_1a_7x7_conv (Conv3D)     (None, 125, 112, 112 65856       input_1[0][0]                    
__________________________________________________________________________________________________
Conv3d_1a_7x7_bn (BatchNormaliz (None, 125, 112, 112 192         Conv3d_1a_7x7_conv[0][0]         
__________________________________________________________________________________________________
Conv3d_1a_7x7 (Activation)      (None, 125, 112, 112 0           Conv3d_1a_7x7_bn[0][0]           
__________________________________________________________________________________________________
MaxPool2d_

In [38]:
pred = rgb_model.predict(X_train[0,:,:,:,:].reshape(1,250,224,224,3))

In [39]:
pred.shape

(1, 31, 1, 1, 1024)

In [42]:
pred3 = rgb_model.predict(X_train[0:3,:,:,:,:])

In [43]:
pred3.shape

(3, 31, 1, 1, 1024)

In [46]:
# snippet to compute TVL1 flow
def compute_TVL1(prev, curr, bound=15):
    """Compute the TV-L1 optical flow."""
    TVL1 = cv2.DualTVL1OpticalFlow_create()
    flow = TVL1.calc(prev, curr, None)
    assert flow.dtype == np.float32

    flow = (flow + bound) * (255.0 / (2*bound))
    flow = np.round(flow).astype(int)
    flow[flow >= 255] = 255
    flow[flow <= 0] = 0

    return flow

In [82]:
import video_generator

In [98]:
test_dir = '../SoccerNet-code/data/test'
train_dir = '../SoccerNet-code/data/train'
dims = (250,224,224,3)
batch_size = 8
videogen = video_generator.VideoGenerator(train_dir, test_dir, dims, batch_size)

In [118]:
training_generator = videogen.generate(train_or_test='train')
training_steps_per_epoch = len(videogen.filenames_train) // batch_size
testing_generator = videogen.generate(train_or_test="test")
testing_steps_per_epoch = len(videogen.filenames_test) // batch_size

Single stream 3D convolution model in RGB channel using ImageNet/Kinetics weights

In [104]:
def RGB_model(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_CLASSES, dropout_prob):

    rgb_model = Inception_Inflated3d(
        include_top=False,
        weights='rgb_imagenet_and_kinetics',
        input_shape=(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, 3))

    x1 = rgb_model.layers[-1].output
    x1 = Dropout(dropout_prob)(x1)

    x1 = generate_logit(x1, '1x1_Conv3d_rgb_logits', NUM_CLASSES)

    x = Activation('softmax', name='prediction')(x1)

    model = Model(input=rgb_model.input, output=x)

    return model

In [105]:
NUM_FRAMES=250
FRAME_HEIGHT=224
FRAME_WIDTH=224
NUM_CLASSES=2
dropout_prob=0.5

In [106]:
test_model = RGB_model(NUM_FRAMES, FRAME_HEIGHT, FRAME_WIDTH, NUM_CLASSES, dropout_prob)

/home/dshin1331/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  from ipykernel import kernelapp as app


In [108]:
from keras.optimizers import adam

In [109]:
test_model.compile(loss='categorical_crossentropy',
              optimizer=adam(),
              metrics=['accuracy'])

In [ ]:
test_model.predict()

In [121]:
for v in training_generator:
    vid =v
    break

In [125]:
vid[0][0].shape

(250, 224, 224, 3)

In [ ]:
test_model.predict(vid[0])

In [110]:
test_model.fit_generator(training_generator,
                                 steps_per_epoch=training_steps_per_epoch,
                         epochs=2,
                         validation_data=testing_generator,
                         validation_steps=testing_steps_per_epoch)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2


KeyboardInterrupt: 